In [12]:
!git config --global user.email "buvoligina@gmail.com"
!git config --global user.name "gbuvoli"
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/tdg_datasets/SaberPRO_project_master
!git remote set-url origin https://github.com/gbuvoli/SaberPRO_project.git

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/tdg_datasets/SaberPRO_project_master


In [13]:
%cd /content/drive/MyDrive/tdg_datasets/SaberPRO_project_master
%run myfunctions.py

/content/drive/MyDrive/tdg_datasets/SaberPRO_project_master


In [15]:
!git add .
!git commit -m "Mensaje descriptivo de tus cambios"
!git push origin master

On branch master
Your branch is ahead of 'origin/master' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean
fatal: could not read Username for 'https://github.com': No such device or address


In [6]:
#echo: false
#|include: false
from myfunctions import *
myimports
import gc

In [7]:
# Configurar Seaborn para que use la paleta "viridis" como predeterminado
sns.set_palette("viridis")

# Configurar el estilo de Seaborn (opcional, para mantener un estilo uniforme)
sns.set_style("whitegrid")

# Ajustar el tamaño de las figuras predeterminado (opcional)
plt.rcParams["figure.figsize"] = (10, 6)

# Establecer opciones para mostrar todas las filas y columnas sin truncar
pd.options.display.max_rows = None  # Muestra todas las filas
pd.options.display.max_columns = None  # Muestra todas las columnas



Cargamos los datasets con la población total de estudiantes que presentaron la prueba SABER PRO entre 2016 y 2024, su información sociodemográfica y sus resultados, sin embargo, es necesario combinarlos ambos para obtener mayor información sociodemográfica y de resultados de las pruebas.

In [8]:
data2018_24=pd.read_csv('/content/drive/MyDrive/tdg_datasets/Saber_Pro_20241022.csv')
data2016_22 =pd.read_csv('/content/drive/MyDrive/tdg_datasets/poblacion2016-2022.csv')
data2016_22.head()

,PERIODO,ESTU_CONSECUTIVO,ESTU_CODDANE_COLE_TERMINO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_COMOCAPACITOEXAMENSB11,ESTU_SEMESTRECURSA,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_HORASSEMANATRABAJA,INST_COD_INSTITUCION,INST_NOMBRE_INSTITUCION,ESTU_PRGM_ACADEMICO,ESTU_SNIES_PRGMACADEMICO,GRUPOREFERENCIA,ESTU_PRGM_CODMUNICIPIO,ESTU_PRGM_DEPARTAMENTO,ESTU_NUCLEO_PREGRADO,ESTU_INST_CODMUNICIPIO,ESTU_INST_DEPARTAMENTO,INST_CARACTER_ACADEMICO,MOD_RAZONA_CUANTITAT_PUNT,MOD_RAZONA_CUANTITAT_DESEM,MOD_LECTURA_CRITICA_PUNT,MOD_LECTURA_CRITICA_DESEM,MOD_COMPETEN_CIUDADA_PUNT,MOD_COMPETEN_CIUDADA_DESEM,MOD_INGLES_PUNT,MOD_INGLES_DESEM,MOD_COMUNI_ESCRITA_PUNT,MOD_COMUNI_ESCRITA_DESEM,PUNT_GLOBAL,filter_$,ZPUNT_GLOBAL,ZSco01
0,20203.0,EK202030341029,NaN,NaN,NaN,NaN,Secundaria (Bachillerato) completa,Educación profesional incompleta,Entre 11 y 20 horas,1202.0,UNIVERSIDAD DEL ATLANTICO-BARRANQUILLA,LICENCIATURA EN HUMANIDADES Y LENGUA CASTELLANA,106170.0,EDUCACIÓN,8001.0,ATLÁNTICO,EDUCACIÓN,8001.0,ATLÁNTICO,UNIVERSIDAD,171.0,3.0,171.0,3.0,154.0,2.0,169.0,A2,131.0,2.0,159.0,Not Selected,0.462851,0.822211
1,20203.0,EK202030286711,NaN,NaN,NaN,NaN,Técnica o tecnológica incompleta,Técnica o tecnológica incompleta,Entre 21 y 30 horas,1202.0,UNIVERSIDAD DEL ATLANTICO-BARRANQUILLA,LICENCIATURA EN HUMANIDADES Y LENGUA CASTELLANA,106170.0,EDUCACIÓN,8001.0,ATLÁNTICO,EDUCACIÓN,8001.0,ATLÁNTICO,UNIVERSIDAD,149.0,2.0,166.0,3.0,158.0,3.0,134.0,A1,165.0,3.0,154.0,Not Selected,0.260342,0.616542
2,20203.0,EK202030340888,NaN,NaN,NaN,NaN,Educación profesional incompleta,Técnica o tecnológica incompleta,Más de 30 horas,1202.0,UNIVERSIDAD DEL ATLANTICO-BARRANQUILLA,LICENCIATURA EN HUMANIDADES Y LENGUA CASTELLANA,106170.0,EDUCACIÓN,8001.0,ATLÁNTICO,EDUCACIÓN,8001.0,ATLÁNTICO,UNIVERSIDAD,159.0,3.0,177.0,3.0,153.0,2.0,161.0,A2,135.0,2.0,157.0,Not Selected,0.381847,0.739944
3,20203.0,EK202030233707,NaN,NaN,NaN,NaN,Primaria incompleta,Primaria incompleta,Menos de 10 horas,1202.0,UNIVERSIDAD DEL ATLANTICO-BARRANQUILLA,LICENCIATURA EN HUMANIDADES Y LENGUA CASTELLANA,106170.0,EDUCACIÓN,8001.0,ATLÁNTICO,EDUCACIÓN,8001.0,ATLÁNTICO,UNIVERSIDAD,118.0,1.0,124.0,1.0,131.0,2.0,101.0,A-,119.0,2.0,119.0,Not Selected,-1.157215,-0.823144
4,20203.0,EK202030312054,NaN,Menos de 500 mil,Repasó por cuenta propia,8.0,Técnica o tecnológica completa,Técnica o tecnológica completa,Entre 11 y 20 horas,1202.0,UNIVERSIDAD DEL ATLANTICO-BARRANQUILLA,LICENCIATURA EN HUMANIDADES Y LENGUA CASTELLANA,106170.0,EDUCACIÓN,8001.0,ATLÁNTICO,EDUCACIÓN,8001.0,ATLÁNTICO,UNIVERSIDAD,149.0,2.0,161.0,3.0,191.0,3.0,167.0,A2,0.0,NaN,134.0,Not Selected,-0.549690,-0.206136


In [ ]:
data2018_24.head()

In [ ]:
# Seleccionamos todas las columnas de df1 y excluimos las duplicadas de df2
df1_columns = data2016_22.columns
df2_columns = data2018_24.columns.difference(df1_columns).to_list() + ['ESTU_CONSECUTIVO']  # Conservamos la clave para el merge

# Realizamos el merge usando 'ESTU_CONSECUTIVO' como clave y un inner join
df_combined = pd.merge(data2016_22[df1_columns], data2018_24[df2_columns], on='ESTU_CONSECUTIVO', how='inner')
df_combined.to_csv('df_combined.csv', index=False)
# Verificamos el resultado
print(df_combined.head())
print("Número de registros combinados:", df_combined.shape[0])
print("Número de columnas en el dataset combinado:", df_combined.shape[1])



In [ ]:
df_combined[df_combined['ESTU_CONSECUTIVO']=='EK202250165035']

In [ ]:
uninorte= df_combined[df_combined['INST_NOMBRE_INSTITUCION']=='UNIVERSIDAD DEL NORTE-BARRANQUILLA']
uninorte.to_csv('data_uninorte_2016-2024.csv', index=False)

print(uninorte.shape)
uninorte.sample(5)

## EXPLORATORY DATA ANALYSIS

Conozcamos la naturaleza de los datos de cada columna y cuantas categorías tienen

In [ ]:
print(f'los datos de poblacion son de tamaño {df_combined.shape}')
print(f'la muestra de UNINORTE es de tamaño {uninorte.shape}')

In [ ]:
# Supongamos que tienes varios DataFrames grandes que ya no necesitas
del data2016_22, data2018_24
gc.collect()  # Forzar la liberación de memoria


Conociendo los datos del dataset general

In [ ]:
df_combined.describe(include='object')

## Evaluación y Manejo de Nulos

In [ ]:
sns.heatmap(df_combined.isnull(), cbar=False)

In [ ]:
df_combined.drop(['ZPUNT_GLOBAL','ESTU_CODDANE_COLE_TERMINO','filter_$','ZSco01' ], axis=1, inplace=True)

In [ ]:
df_combined.isnull().sum()/df_combined.shape[0]*100

In [ ]:
keep_float=['PUNT_GLOBAL','MOD_COMUNI_ESCRITA_PUNT','MOD_COMPETEN_CIUDADA_PUNT','MOD_LECTURA_CRITICA_PUNT','MOD_RAZONA_CUANTITAT_PUNT','MOD_INGLES_PUNT']
for col in df_combined.columns:
    if col not in keep_float:
        df_combined.fillna({col:'sin_informacion'}, inplace=True)
        df_combined[col]=df_combined[col].astype(str)

In [ ]:
df_combined.isnull().sum()/df_combined.shape[0]*100

In [ ]:
df_combined.dtypes

In [ ]:
cols_low=[]
cols_hight=[]
cols_unica=[]

for col in df_combined.columns:
    if df_combined[col].nunique()==1:
        cols_unica.append(col)
        print(f'La columna {col} tiene categoría unica')
    elif df_combined[col].nunique()<100:
        cols_low.append(col)
        print(df_combined[col].value_counts())
    else:
        cols_hight.append(col)
        print(f'La columna {col} tiene mas de 100 categorías')

print(f'El dataset tiene {len(cols_low)} columnas de baja frecuencia')
print(f'El dataset tiene {len(cols_hight)} columnas de alta frecuencia')
print(f'El dataset tiene {len(cols_unica)} columnas con valores unicos')


In [ ]:
uninorte=pd.read_csv('data_uninorte_2016-2024.csv')
uninorte.head()

In [ ]:
keep_float=['PUNT_GLOBAL','MOD_COMUNI_ESCRITA_PUNT','MOD_COMPETEN_CIUDADA_PUNT','MOD_LECTURA_CRITICA_PUNT','MOD_RAZONA_CUANTITAT_PUNT','MOD_INGLES_PUNT']
for col in uninorte.columns:
    if col not in keep_float:
        uninorte.fillna({col:'sin_informacion'}, inplace=True)
        uninorte[col]=uninorte[col].astype(str)

In [ ]:
# Obtener las categorías únicas excluyendo nulos y convertirlas a strings
category_order = sorted(df_combined['FAMI_ESTRATOVIVIENDA'].dropna().astype(str).unique())

# Crear un diccionario de colores para cada categoría
palette_dict = dict(zip(category_order, sns.color_palette("viridis", len(category_order))))
fig, ax = plt.subplots(1, 2, figsize=(12, 6), sharey=False)

# Gráfico para Uninorte
sns.countplot(data=uninorte, x='ESTU_GENERO', hue='ESTU_GENERO', legend=False,  ax=ax[0])
ax[0].set_title('Distribución de Género - Uninorte')

# Gráfico para la población general
sns.countplot(data=df_combined, x='ESTU_GENERO',hue='ESTU_GENERO', legend=False,  ax=ax[1])
ax[1].set_title('Distribución de Género - Población General')

plt.tight_layout()
plt.show()


In [ ]:
# Obtener las categorías únicas excluyendo nulos y convertirlas a strings
category_order = sorted(df_combined['FAMI_ESTRATOVIVIENDA'].unique())

# Crear un diccionario de colores para cada categoría
palette_dict = dict(zip(category_order, sns.color_palette("viridis", len(category_order))))

fig, ax = plt.subplots(1, 2, figsize=(12, 6), sharey=False)

# Gráfico para Uninorte
sns.countplot(data=uninorte, x='FAMI_ESTRATOVIVIENDA', hue='FAMI_ESTRATOVIVIENDA',
              order=category_order, palette=palette_dict, ax=ax[0])
ax[0].set_title('Estrato Socioeconómico - Uninorte')
ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=45, ha="right")  # Gira las etiquetas del eje x

# Gráfico para la población general
sns.countplot(data=df_combined, x='FAMI_ESTRATOVIVIENDA', hue='FAMI_ESTRATOVIVIENDA',
              order=category_order, palette=palette_dict, ax=ax[1])
ax[1].set_title('Estrato Socioeconómico - Población General')
ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=45, ha="right")  # Gira las etiquetas del eje x

plt.tight_layout()
plt.show()




In [ ]:
# Obtener las categorías únicas excluyendo nulos y convertirlas a strings
category_order = sorted(df_combined['FAMI_EDUCACIONPADRE'].unique())

# Crear un diccionario de colores para cada categoría
palette_dict = dict(zip(category_order, sns.color_palette("viridis", len(category_order))))

fig, ax = plt.subplots(1, 2, figsize=(12, 6), sharey=False)

# Gráfico para Uninorte
sns.countplot(data=uninorte, x='FAMI_EDUCACIONPADRE', hue='FAMI_EDUCACIONPADRE',
              order=category_order, palette=palette_dict, ax=ax[0])
ax[0].set_title('Nivel educativo PADRE- Uninorte')
ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=45, ha="right")  # Gira las etiquetas del eje x

# Gráfico para la población general
sns.countplot(data=df_combined, x='FAMI_EDUCACIONPADRE', hue='FAMI_EDUCACIONPADRE',
              order=category_order, palette=palette_dict, ax=ax[1])
ax[1].set_title('Nivel educativo PADRE - Población General')
ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=45, ha="right")  # Gira las etiquetas del eje x

plt.tight_layout()
plt.show()


In [ ]:
# Obtener las categorías únicas excluyendo nulos y convertirlas a strings
category_order = sorted(df_combined['FAMI_EDUCACIONMADRE'].unique())

# Crear un diccionario de colores para cada categoría
palette_dict = dict(zip(category_order, sns.color_palette("viridis", len(category_order))))

fig, ax = plt.subplots(1, 2, figsize=(12, 6), sharey=False)

# Gráfico para Uninorte
sns.countplot(data=uninorte, x='FAMI_EDUCACIONMADRE', hue='FAMI_EDUCACIONMADRE',
              order=category_order, palette=palette_dict, ax=ax[0])
ax[0].set_title('Nivel educativo MADRE- Uninorte')
ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=45, ha="right")  # Gira las etiquetas del eje x

# Gráfico para la población general
sns.countplot(data=df_combined, x='FAMI_EDUCACIONMADRE', hue='FAMI_EDUCACIONMADRE',
              order=category_order, palette=palette_dict, ax=ax[1])
ax[1].set_title('Nivel educativo MADRE - Población General')
ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=45, ha="right")  # Gira las etiquetas del eje x

plt.tight_layout()
plt.show()


In [ ]:
# Obtener las categorías únicas excluyendo nulos y convertirlas a strings
category_order = sorted(df_combined['ESTU_HORASSEMANATRABAJA'].unique())

# Crear un diccionario de colores para cada categoría
palette_dict = dict(zip(category_order, sns.color_palette("viridis", len(category_order))))

fig, ax = plt.subplots(1, 2, figsize=(12, 6), sharey=False)

# Gráfico para Uninorte
sns.countplot(data=uninorte, x='ESTU_HORASSEMANATRABAJA', hue='ESTU_HORASSEMANATRABAJA',
              order=category_order, palette=palette_dict, ax=ax[0])
ax[0].set_title('Horas a la semana que TRABAJA- Uninorte')
ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=45, ha="right")  # Gira las etiquetas del eje x

# Gráfico para la población general
sns.countplot(data=df_combined, x='ESTU_HORASSEMANATRABAJA', hue='ESTU_HORASSEMANATRABAJA',
              order=category_order, palette=palette_dict, ax=ax[1])
ax[1].set_title('Horas a la semana que TRABAJA - Población General')
ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=45, ha="right")  # Gira las etiquetas del eje x

plt.tight_layout()
plt.show()

In [ ]:
# Extraer el año de PERIODO
uninorte['AÑO'] = uninorte['PERIODO'].astype(str).str[:4].astype(int)
df_combined['AÑO'] = df_combined['PERIODO'].astype(str).str[:4].astype(int)

# Lista de las categorías de horas trabajadas
category_order = sorted(df_combined['ESTU_HORASSEMANATRABAJA'].dropna().unique())

# Configurar el tamaño de los gráficos
fig, ax = plt.subplots(2, 1, figsize=(12, 12), sharex=True)

# Gráfico para Uninorte
uninorte_proportions = uninorte.groupby(['AÑO', 'ESTU_HORASSEMANATRABAJA']).size().unstack(fill_value=0)
uninorte_proportions = uninorte_proportions.div(uninorte_proportions.sum(axis=1), axis=0)

# Graficar cada categoría de horas trabajadas como una línea en el tiempo
for category in category_order:
    if category in uninorte_proportions.columns:
        sns.lineplot(x=uninorte_proportions.index, y=uninorte_proportions[category], label=str(category), ax=ax[0])
ax[0].set_title('Proporción de Horas Trabajadas - Uninorte')
ax[0].set_ylabel('Proporción')
ax[0].legend(title="Horas Trabajadas", bbox_to_anchor=(1.05, 1), loc='upper left')
ax[0].grid(True)

# Gráfico para la población general
general_proportions = df_combined.groupby(['AÑO', 'ESTU_HORASSEMANATRABAJA']).size().unstack(fill_value=0)
general_proportions = general_proportions.div(general_proportions.sum(axis=1), axis=0)

# Graficar cada categoría de horas trabajadas como una línea en el tiempo
for category in category_order:
    if category in general_proportions.columns:
        sns.lineplot(x=general_proportions.index, y=general_proportions[category], label=str(category), ax=ax[1])
ax[1].set_title('Proporción de Horas Trabajadas - Población General')
ax[1].set_xlabel('Año')
ax[1].set_ylabel('Proporción')
ax[1].legend(title="Horas Trabajadas", bbox_to_anchor=(1.05, 1), loc='upper left')
ax[1].grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# Lista de las columnas de puntajes de las pruebas y del puntaje global
pruebas = [
    'MOD_RAZONA_CUANTITAT_PUNT', 'MOD_COMUNI_ESCRITA_PUNT',
    'MOD_COMPETEN_CIUDADA_PUNT', 'MOD_LECTURA_CRITICA_PUNT',
    'MOD_INGLES_PUNT', 'PUNT_GLOBAL'
]

# Iteramos sobre cada columna y generamos un gráfico KDE
for prueba in pruebas:
    plt.figure(figsize=(12, 6))
    sns.kdeplot(data=uninorte, x=prueba, label='Uninorte', fill=True)
    sns.kdeplot(data=df_combined, x=prueba, label='Población General', fill=True)
    plt.title(f'Distribución de {prueba}')
    plt.xlabel('Puntaje')
    plt.ylabel('Densidad')
    plt.legend()
    plt.show()


In [ ]:
# Extraer el año de PERIODO
df_combined['AÑO'] = df_combined['PERIODO'].astype(str).str[:4].astype(int)
uninorte['AÑO']= uninorte['PERIODO'].astype(str).str[:4].astype(int)


# Crear gráficos de series temporales por cada prueba
for prueba in pruebas:
    plt.figure(figsize=(12, 6))

    # Calcular promedio de puntaje por AÑO para Uninorte y población general
    uninorte_mean = uninorte.groupby('AÑO')[prueba].mean()
    general_mean = df_combined.groupby('AÑO')[prueba].mean()

    # Graficar series temporales
    sns.lineplot(data=uninorte_mean, label='Uninorte', marker='o')
    sns.lineplot(data=general_mean, label='Población General', marker='o')

    # Configuración del gráfico
    plt.title(f'Serie Temporal Anual de {prueba}')
    plt.xlabel('Año')
    plt.ylabel('Puntaje Promedio')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.show()



In [ ]:
df_combined.columns

## Análisis de correspondencia multiple

In [ ]:
import prince

# Seleccionar solo las variables categóricas
categorical_cols = df_combined.select_dtypes(include=['object']).columns

# Llenar valores nulos con una categoría temporal
#

# Aplicar MCA
mca = prince.MCA(n_components=10)
mca = mca.fit(df_categorical)

# Graficar los primeros dos componentes del MCA
mca.plot_coordinates(X=df_categorical, show_row_points=False, show_column_labels=True)
plt.title('Análisis de Correspondencia Múltiple (MCA)')
plt.show()
